### スパッタリングシミュレーションで原子が基盤の下から抜けていないか確認するスクリプト

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [19]:
path2target = "/home/k0472/k047207/result/lammps/Si/sputtering/ml/depth15/model18/0deg_1000eV"

In [20]:
def get_num_sputtered_injected_atoms(path2target, insert_atom_every=20000):
    with open(os.path.join(path2target, 'etch.dat'), mode='r') as f:
        lines = [s.strip() for s in f.readlines()]
    idx_list = []
    for i, l in enumerate(lines):
        if 'ITEM: TIMESTEP' in l:
            idx_list.append(i)

    idx_list_shift = idx_list[1:]
    idx_list_shift.append(len(lines))
    block = []
    for idx1, idx2 in zip(idx_list, idx_list_shift):
        block.append(lines[idx1:idx2])

    result = []
    timestep_idx = 1
    num_sputter_atom_idx = 3
    for b in block:
        timestep = int(b[timestep_idx])
        num_sputter_atom = int(b[num_sputter_atom_idx])
        result.append(np.array([timestep, num_sputter_atom]))
    result = np.array(result)

    total_num_sputter_atom = result[:,1].sum()
    last_timestep = int(block[-1][timestep_idx])
    def get_num_insert_atom(insert_atom_every, last_timestep):
        return last_timestep // insert_atom_every
    
    total_num_insert_atom = get_num_insert_atom(insert_atom_every, last_timestep)

    return total_num_sputter_atom, total_num_insert_atom

In [21]:
total_num_sputter_atom, total_num_insert_atom = get_num_sputtered_injected_atoms(path2target)

In [22]:
print("total_num_sputter_atom: ", total_num_sputter_atom)
print("total_num_insert_atom: ", total_num_insert_atom)

total_num_sputter_atom:  713
total_num_insert_atom:  1000


In [23]:
with open(os.path.join(path2target, 'log.lammps'), mode='r') as f:
    lines = [s.strip() for s in f.readlines()]
num_atom_list = list(filter(lambda s: 'Loop time' in s, lines))

init_num_atom = int(num_atom_list[0].split(' ')[-2])
final_num_atom = int(num_atom_list[-1].split(' ')[-2])
final_num_atom_expected = init_num_atom - total_num_sputter_atom + total_num_insert_atom

In [24]:
print("初期構造の原子数: ", init_num_atom)
print("注入原子数", total_num_insert_atom)
print("スパッタした原子数", total_num_sputter_atom)

print("想定される最終構造の原子数: ", final_num_atom_expected)
print("最終構造の原子数: ", final_num_atom)
if final_num_atom_expected == final_num_atom:
    print("-> 原子は突き抜けていない")
else:
    num_passed_atom = final_num_atom_expected - final_num_atom
    print(f"-> {num_passed_atom}原子が突き抜けている")

初期構造の原子数:  11250
注入原子数 1000
スパッタした原子数 713
想定される最終構造の原子数:  11537
最終構造の原子数:  11535
-> 2原子が突き抜けている


In [25]:
num_atom_list = list(filter(lambda s: 'Loop time' in s, lines))
init_num_atom = int(num_atom_list[0].split(' ')[-2])
final_num_atom = int(num_atom_list[-1].split(' ')[-2])

In [26]:
init_num_atom

11250